In [3]:
import pandas as pd
import numpy as np
import os, json

from os import listdir
from fuzzywuzzy import fuzz

In [4]:
DATAPATH = 'data'
VERSION = 'cord19_v20'
META_PATH ="/".join([DATAPATH, VERSION, 'metadata.csv'])

In [5]:
meta_df = pd.read_csv(META_PATH)
print(meta_df.shape)
# meta_df.head()

(63571, 18)


C:\Users\Khachatur\Miniconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
cuid_dup = meta_df[meta_df.duplicated(['cord_uid'])]
cuid_dup_set = set(cuid_dup['cord_uid'])
print("All cuid duplicate records:"len(cuid_dup))

SyntaxError: invalid syntax (<ipython-input-6-a53374bc6351>, line 3)

In [ ]:
for uid in cuid_dup_set:
    dup_df = meta_df[meta_df['cord_uid'] == uid]
    if len(dup_df['pubmed_id'].unique()) != 1 and len(dup_df['doi'].unique()) != 1:
        print(dup_df['pubmed_id'].unique())
        print(dup_df['doi'].unique())

In [ ]:
has_pdfpar_df = meta_df[meta_df['pdf_json_files'].isna() == False]
pdfpar_dup = has_pdfpar_df[has_pdfpar_df.duplicated(['pdf_json_files'])]
print("Duplicte pdf paths:", len(pdfpar_dup))
print("Total pdf paths", len(has_pdfpar_df))

In [ ]:
has_pmcpar_df = meta_df[meta_df['pmc_json_files'].isna() == False]
pmcpar_dup = has_pmcpar_df[has_pmcpar_df.duplicated(['pmc_json_files'])]
print("Duplicte pmc paths:", len(pmcpar_dup))
print("Total pmc paths", len(has_pmcpar_df))

In [7]:
has_sha_df = meta_df[meta_df['sha'].isna() == False]
mult_sha_df = has_sha_df[has_sha_df['sha'].str.len() != 40]
sing_sha_df = has_sha_df[has_sha_df['sha'].str.len() == 40]
print("# of records with multiple associated sha values:", len(mult_sha_df))

# of records with multiple associated sha values: 2517


In [8]:
has_pmc_df = meta_df[meta_df['pmcid'].isna() == False]
print("# of weird pmcid values:", len(has_pmc_df[(has_pmc_df['pmcid'].str.len() != 9) & (has_pmc_df['pmcid'].str.len() != 10) & (has_pmc_df['pmcid'].str.len() != 8)]))

# of weird pmcid values: 0


In [9]:
from os import listdir
PARSE_PATH ="/".join([DATAPATH, VERSION, 'document_parses'])

In [10]:
pdf_path = "/".join([PARSE_PATH, 'pdf_json'])
pmc_path = "/".join([PARSE_PATH, 'pmc_json'])
pdf_parses = listdir(pdf_path)
pmc_parses = listdir(pmc_path)
print("Total # pdfs:", len(pmc_parses))
print("Total # pmcs:", len(pdf_parses))

Total # pdfs: 33503
Total # pmcs: 51868


In [11]:
sha_list = []
for row in mult_sha_df.iterrows():
    new_l = row[1].sha.split("; ")
    sha_list.extend(new_l)
print("All sha values in multi sha record list:", len(sha_list))
print("All UNIQUE sha values in multi sha record list:", len(set(sha_list)))

print("All sha values in single sha record list:", len(list(sing_sha_df.sha)))
print("All UNIQUE sha values in single sha record list:", len(set(sing_sha_df.sha)))

sha_list.extend(list(sing_sha_df.sha))
print("All sha values in combined sha record list:", len(sha_list))
print("All UNIQUE sha values in combined sha record list:", len(set(sha_list)))
total_unique_sha = len(set(sha_list))

All sha values in multi sha record list: 5475
All UNIQUE sha values in multi sha record list: 5475
All sha values in single sha record list: 46407
All UNIQUE sha values in single sha record list: 46394
All sha values in combined sha record list: 51882
All UNIQUE sha values in combined sha record list: 51868


In [12]:
has_path_df = meta_df[(meta_df["pdf_json_files"].isna() == False) | (meta_df["pmc_json_files"].isna() == False)]
print("Total full text records:", len(has_path_df))

Total full text records: 49862


In [14]:
errors_id = []
errors_title = []
for row in has_path_df.iterrows():
    record = row[1]
    data = None
    if not isinstance(record.pmc_json_files, float):
        json_paths = ["/".join([DATAPATH, VERSION, record.pmc_json_files])]
        p_ids = [record.pmcid]
    else:
        pdf_paths = record.pdf_json_files.split("; ")
        json_paths = ["/".join([DATAPATH, VERSION, x]) for x in pdf_paths]
        p_ids = record.sha.split("; ")
    
    for i, path in enumerate(json_paths):
        cur_size = len(errors_title)
        with open(path) as f:
            data = json.load(f)

        if data['paper_id'] != p_ids[i]:
            
            errors_id.append([data['paper_id'], p_ids[i]])
            
        json_title = data['metadata']['title'].lower()
        try:
            meta_title = record.title.lower()
        except:
            meta_title = ''
        title_ratio = fuzz.ratio(meta_title, json_title)
        partial_ratio = fuzz.partial_ratio(meta_title, json_title)

        if cur_size == len(errors_title):
            if title_ratio > 85 or partial_ratio > 85:
                break
            else:
                errors_title.append([json_title, meta_title, title_ratio, partial_ratio, data['paper_id']])
        else:
            if title_ratio > 85 or partial_ratio > 85:
                errors_title.pop()
                break
            elif title_ratio > errors_title[-1][3] or partial_ration > errors_title[-1][3]:
                errors_title.pop()
                errors_title.append([json_title, meta_title, title_ratio, partial_ratio, data['paper_id']])

In [15]:
empty_json_titles = 0
empty_meta_titles = 0
other_errors_titles = 0
sha_errors = 0
pmc_errors = 0
for error in errors_title:
    if error[0] == '':
        empty_json_titles += 1
    if error[1] == '':
        empty_meta_titles += 1
    if error[0] != '' and error[1] !='':
        other_errors_titles += 1
    if len(error[4]) == 40:
        sha_errors += 1
    if len(error[4]) != 40:
        pmc_errors += 1

In [17]:
print("Total ID errors:", len(errors_id))
print("Discrepencies between metadata title and file title:", len(errors_title))
print("Empty title value in json file:", empty_json_titles)
print("Empty title value from metadata records:", empty_meta_titles)
print("All other title errors:", other_errors_titles)
print("Errors in pdf files:", sha_errors)
print("Errors in pmc files:", pmc_errors)

Total ID errors: 0
Discrepencies between metadata title and file title: 3547
Empty title value in json file: 2294
Empty title value from metadata records: 6
All other title errors: 1247
Errors in pdf files: 3527
Errors in pmc files: 20
